# Building a Project Manager (PM) Agent for VM Migration in Python

## What You'll Learn

In this notebook, you will learn how to build and deploy a Project Manager (PM) agent designed to manage the execution of a Virtual Machine (VM) migration plan. This agent will coordinate various tasks, monitor progress, and dynamically adapt the plan based on feedback from different engineering agents. By the end of this notebook, you will understand how to:

- Set up and configure a model to support the PM agent's decision-making process.
- Define and manage the state for the PM agent to track task progress and dependencies.
- Implement and customize the PM agent to handle specific responsibilities within the VM migration workflow.
- Construct and compile a workflow graph to represent the PM agent’s interactions with other agents.
- Execute the workflow, manage outputs, and handle dynamic adjustments to the plan based on real-time feedback.

## Basic Concepts

Before diving into the implementation, it's essential to understand some basic concepts:

- **PM Agent:** A specialized agent that autonomously manages the execution of a VM migration plan. The PM agent coordinates tasks, assigns responsibilities to other agents (e.g., OCP Engineer, vSphere Engineer), and tracks the overall progress of the migration.

- **State:** A shared data structure that stores the context, task statuses, dependencies, and other critical information required by the PM agent. Effective state management ensures that the PM agent can track progress, handle feedback, and adapt the plan as needed.

- **VM Migration Plan:** A structured set of tasks required to move virtual machines from one environment to another. The PM agent will break down the migration plan into individual tasks, assign them to the appropriate agents, and ensure they are executed in the correct order.

- **Workflow Graph:** A structured representation of the PM agent's workflow, where nodes represent tasks or decisions, and edges define the flow of control between these steps. This graph helps visualize and manage the sequence of actions during the VM migration process.

- **Feedback Handling:** The process by which the PM agent receives and processes feedback from other agents (e.g., task completion, errors, or issues) and adapts the migration plan accordingly to ensure a smooth execution.

- **Prompt Engineering:** The process of crafting prompts that guide the PM agent's interaction with the model. Proper prompt engineering ensures that the agent generates relevant and accurate tasks and adjustments to the migration plan.

Understanding these concepts will provide a solid foundation as we proceed with the practical implementation of the PM agent and its role in managing a VM migration plan.


## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [89]:
%pip install langgraph langgraph-checkpoint-sqlite requests jsonschema tenacity 

Note: you may need to restart the kernel to use updated packages.


## 2. Building Blocks

In this section, we'll start by building the basic components that our agent will use. These building blocks will form the foundation of our agent, enabling it to keep track of time, store data, and interact with a model.

### Datetime Function

The first building block we'll create is a simple function to get the current time. This is important because our agent might need to timestamp certain actions or events. Let's write a function that returns the current date and time in UTC format:


In [90]:
from datetime import datetime, timezone


def get_current_utc_datetime():
    now_utc = datetime.now(timezone.utc)
    return now_utc.strftime("%Y-%m-%d %H:%M:%S.%f UTC")[:-3]


# Example usage:
print("Current UTC datetime:", get_current_utc_datetime())

Current UTC datetime: 2024-09-04 17:55:39.378377 


### Basic Agent Data Handling
Next, we'll introduce a simple way to handle data for our agent. In this case, we'll create a function that allows us to load and manage the agent's data. This data might include things like the agent's responsibilities, the tasks it needs to perform, and other relevant information.

Let's assume the data is stored in a YAML file (a common format for configuration files), and we'll write a function to load this data:

In [91]:
import yaml
from typing import Dict, Any


def load_agent_descriptions(description_file: str) -> Dict[str, Any]:
    """
    Load the agent descriptions from a YAML file.
    """
    try:
        with open(description_file, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"Description file '{description_file}' not found.")


# Example usage:
agents_description = load_agent_descriptions("agents.yaml")
print("Loaded agent descriptions:", agents_description)

Loaded agent descriptions: {'agents': [{'name': 'Planner Agent', 'role': 'Planner Agent', 'responsibilities': ['Creates a comprehensive Migration Plan based on the tutorial.', 'Identifies key steps, target VMs, and source/target providers.', 'Coordinates and structures the plan for execution by other agents.']}, {'name': 'Project Manager (PM) Agent', 'role': 'Project Manager Agent', 'responsibilities': ['Manages the breakdown of tasks for the migration process.', 'Oversees task execution and ensures agents are working in coordination.', 'Ensures timelines are followed and adjusts the plan as necessary.', 'Communicates with all agents to ensure smooth task progression and resolve bottlenecks.']}, {'name': 'vSphere Engineer Agent', 'role': 'vSphere Engineer Agent', 'responsibilities': ['Handles VM identification and configuration within the vSphere environment.', 'Identifies VMs to migrate based on the tutorial instructions.', 'Manages and configures the Migration Toolkit for Virtualizat

## 3. Configuring a Simple Model
Now that we have the basic building blocks, we'll move on to configuring a model that our agent can use to perform its tasks. This model will process inputs (like a user request) and generate outputs (like a task list).

### Model Configuration
We'll start by setting up a simple configuration for the model. This configuration will include details like the model's endpoint, temperature, and other parameters. Let's create a function to handle this:

In [92]:
def setup_ollama_model(
    model, temperature=0.0, top_p=1.0, top_k=0, repetition_penalty=1.0, stop=None
):
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "repetition_penalty": repetition_penalty,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration:
ollama_config = setup_ollama_model(model="llama3:instruct")
print("Model configuration:", ollama_config)

Model configuration: {'model_endpoint': 'http://localhost:11434/api/generate', 'model': 'llama3:instruct', 'temperature': 0.0, 'top_p': 1.0, 'top_k': 0, 'repetition_penalty': 1.0, 'headers': {'Content-Type': 'application/json'}, 'stop': None}


This function returns a dictionary with the model's configuration. You can adjust the parameters based on the specific model you're using or the task requirements.

### Preparing a Request
With the model configured, the next step is to prepare a request that the agent can send to the model. This request will include the user's input, the system's instructions, and any other necessary information. Let's write a function to prepare this request:

In [93]:
def prepare_payload(
    user_prompt: str,
    sys_prompt: str,
    stream: bool = False,
    config: Dict[str, Any] = ollama_config,
) -> Dict[str, Any]:
    return {
        "model": config.get("model"),
        "format": "json",
        "prompt": user_prompt,
        "system": sys_prompt,
        "stream": stream,
        "temperature": config.get("temperature", 0.0),
        "top_p": config.get("top_p", 1.0),
        "top_k": config.get("top_k", 0),
        "repetition_penalty": config.get("repetition_penalty", 1.0),
        "stop": config.get("stop"),
    }

### Sending a Request
We'll start by writing a function to send the request to the model's endpoint and receive a response:

In [94]:
import requests
import json


def request_model_generate_endpoint(
    payload: Dict[str, Any], config: Dict[str, Any] = ollama_config
) -> Dict[str, Any]:
    try:
        response = requests.post(
            config.get("model_endpoint"),
            headers=config.get("headers", {"Content-Type": "application/json"}),
            data=json.dumps(payload),
            timeout=30,
        )
        response.raise_for_status()

        if response.content.strip():
            return response.json()
        else:
            return {"error": "Empty response from model"}

    except requests.RequestException as e:
        raise Exception(f"Request failed: {e}")

This function sends the prepared payload to the model's endpoint using the `requests` library. It then checks if the response is valid and returns the content. If there's an error in the request, it raises an exception with a descriptive message.

### Processing the Response
Finally, we'll write a function to process and understand the model's response. This might involve formatting the response or extracting specific information:

In [95]:
def process_model_response(response_json: Dict[str, Any]) -> str:
    try:
        response_content = json.loads(response_json.get("response", "{}"))
        pretty_content = json.dumps(response_content, indent=4)

        return pretty_content
    except json.JSONDecodeError:
        return "Error processing the response"

## 4. State Management

State management is an essential concept when working with agents. The "state" of an agent refers to its current condition or the information it has at any given time. For instance, if an agent is working through a list of tasks, its state might include which tasks have been completed, which are in progress, and which are yet to be started.

Managing this state is crucial because it allows the agent to keep track of what it has done and what it needs to do next. Without proper state management, an agent might lose track of its progress, repeat tasks, or skip important steps. In this notebook, you'll learn how to manage an agent's state effectively, ensuring that it operates smoothly and efficiently.

### Implementation of State Management

To implement state management for our PM agent, we define a structured data model (`AgentGraphState`) that holds all the relevant information the agent needs to function effectively. This model includes:
- **Input:** The current command or task that the agent is working on.
- **Response:** The outputs or actions the agent has generated.
- **Feedback:** Any feedback received from other agents or parts of the system.
- **Validation Status:** A flag indicating whether the agent's output has been validated, ensuring that all tasks meet the required criteria before proceeding.

Additionally, we provide a utility function, `update_state`, which allows for updating specific elements of the agent's state. This function ensures that the state is consistently and accurately maintained, which is critical for the agent to operate effectively. By checking for the existence of keys before updating, the function helps prevent errors and maintains the integrity of the state.

Together, these components form the backbone of the agent's state management system, enabling it to manage complex workflows and adapt to changes dynamically.

In [96]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any

class AgentGraphState(TypedDict):
    input: str
    response: Annotated[list, add_messages]
    feedback: Annotated[list, add_messages]
    validated: bool = False


def update_state(state: AgentGraphState, key: str, value: Any):
    """
    Update the state of the agent. Warn if the key doesn't exist.
    """
    if key in state:
        state[key] = value
    else:
        print(f"Warning: Attempting to update a non-existing state key '{key}'.")

## 5. What is an Agent?

In simple terms, an agent is a program that can perform tasks autonomously based on a set of instructions. Think of an agent as a virtual assistant that can handle specific jobs for you. For example, in the context of data science or software development, an agent might process data, make decisions based on that data, and then carry out actions like sending requests or updating records.

Agents are often designed to work without constant human intervention. Once you give them the initial instructions, they can execute tasks on their own, making them very useful in automating repetitive or complex processes.


### System Prompts

System prompts are predefined instructions provided to large language models (LLMs) that guide the AI's behavior during interactions. They set the context, define the role of the AI, and establish rules for how the model should respond to user inputs.

#### Importance of System Prompts

System prompts are essential for ensuring that an AI model like our PM agent performs its tasks consistently and effectively. By specifying the role, task guidelines, and response format, the system prompt helps the AI maintain focus on its objectives—in this case, managing a VM migration plan. 

#### How They Work

When the PM agent receives a query, the system prompt shapes how it processes the input and generates its output. This includes breaking down tasks, assigning them to the correct agents, and handling feedback—all within the structure provided by the prompt.

In summary, system prompts are a powerful tool that directs AI behavior, ensuring responses are aligned with the desired goals and context.


In [97]:
DEFAULT_SYS_PM_PROMPT = """
system

Environment: ipython  
Cutting Knowledge Date: December 2023  
Today Date: {datetime}  

You are the **Project Manager (PM) Agent** responsible for transforming the **Migration Plan Document (MPD)** into an actionable execution plan. Your mission is to ensure the smooth execution of migration tasks by coordinating agents (OCP Engineer, vSphere Engineer, Cleanup) and monitoring progress from start to finish.

---

### Feedback Handling:
If any task encounters issues or feedback, adapt the execution plan dynamically to accommodate the changes. Ensure that all agents are informed accordingly and that tasks are adjusted based on feedback. Here is the feedback received:
Feedback:

{feedback}

---

### Agents Description:
{agents_description}

---

### Guidelines:

1. **Task Status Management:** Monitor feedback from agents (e.g., OCP Engineer, vSphere Engineer) and update the status of existing tasks. Only create new tasks if explicitly required by new information or feedback.
2. **Transform MPD into Tasks:** Break down the MPD into clear, actionable tasks with unique IDs, names, descriptions, assigned agents, dependencies, and acceptance criteria.
3. **Assign Agents:** Assign each task to the correct agent based on the task’s requirements and the agents’ roles.
4. **Track Status:** Continuously monitor task progress and update task statuses as `"pending"`, `"in_progress"`, `"completed"`, or `"failed"`.
5. **Handle Dependencies:** Ensure that tasks only begin after all their dependencies are completed.
6. **Facilitate Communication:** Ensure smooth communication between agents to resolve task dependencies or blockers.
7. **Define Acceptance Criteria:** Ensure each task has clear acceptance criteria, and update task statuses based on the completion of those criteria.

---

### Output Format (Task Structure):

Your response should return a task list in the following format. This task list is central to ensuring that all migration tasks are tracked and executed correctly:

{{
    "tasks": [
        {{
            "task_id": "string",  # Unique identifier for the task.
            "task_name": "string",  # The short name of the task (e.g., "Create Migration Plan").
            "task_description": "string",  # A detailed description of the task's actions.
            "agent": "string",  # The agent responsible for executing the task (must be one of: "ocp_engineer", "vsphere_engineer", "cleanup").
            "status": "pending",  # The current status of the task ("pending", "in_progress", "completed", "failed").
            "dependencies": ["array"],  # Task IDs that must be completed before this task starts.
            "acceptance_criteria": "string",  # The criteria for determining task success (e.g., "Migration plan created and validated").
            "tool_to_use": "string or null",  # The tool that the agent should use to execute the task. This may be null if no tool is required.
            "provided_inputs": {{
                "key": "string | array | null"  # Input data needed for this task, such as VM names or configurations.
            }}
        }}
    ]
}}

---

### Example of a Task List:

{{
    "tasks": [
        {{
            "task_id": "task_001",
            "task_name": "Retrieve VM Details (Database)",
            "task_description": "The vSphere engineer will retrieve the details of the virtual machine named 'database'.",
            "agent": "vsphere_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "VM details retrieved successfully.",
            "tool_to_use": "retrieve_vm_details_tool"
        }},
        {{
            "task_id": "task_002",
            "task_name": "Create Migration Plan",
            "task_description": "Create a migration plan for the specified virtual machines.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "Migration plan created and validated.",
            "tool_to_use": "create_migration_plan_tool",
            "provided_inputs": {{
                "vm_names": ["vm1", "vm2", "vm3"],
                "plan_name": "database-plan"
            }}
        }},
        {{
            "task_id": "task_003",
            "task_name": "Start Migration",
            "task_description": "Start the migration process using the migration plan.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": ["task_001"],
            "acceptance_criteria": "Migration process successfully started.",
            "tool_to_use": "start_migration_tool",
            "provided_inputs": {{
                "plan_name": "database-plan"
            }}
        }}
    ]
}}


---

Remember:
- **Do not change the original task plan** unless explicitly required by new information, dependencies, or feedback.
- Always prioritize updating existing tasks based on feedback before creating new tasks.
- Ensure that the original tasks are followed as closely as possible to avoid unnecessary changes in the execution plan.
- Ensure tasks are marked as complete once their acceptance criteria are met.
- Maintain the JSON format and ensure all fields are filled out correctly, including the `tool_to_use` field with the appropriate tool name.
"""

In [98]:
def write_pm_prompt(
    feedback_value: str = "",
    agents_description: str = agents_description,
) -> str:
    return DEFAULT_SYS_PM_PROMPT.format(
        agents_description=agents_description,
        feedback=feedback_value,
        datetime=get_current_utc_datetime(),
    )

In [99]:
# Example usage:
payload = prepare_payload(
    user_prompt="What tasks should I complete in order to make pasta?",
    sys_prompt=write_pm_prompt(),
)
print("Prepared payload:", payload)

Prepared payload: {'model': 'llama3:instruct', 'format': 'json', 'prompt': 'What tasks should I complete in order to make pasta?', 'system': '\nsystem\n\nEnvironment: ipython  \nCutting Knowledge Date: December 2023  \nToday Date: 2024-09-04 17:55:39.449603   \n\nYou are the **Project Manager (PM) Agent** responsible for transforming the **Migration Plan Document (MPD)** into an actionable execution plan. Your mission is to ensure the smooth execution of migration tasks by coordinating agents (OCP Engineer, vSphere Engineer, Cleanup) and monitoring progress from start to finish.\n\n---\n\n### Feedback Handling:\nIf any task encounters issues or feedback, adapt the execution plan dynamically to accommodate the changes. Ensure that all agents are informed accordingly and that tasks are adjusted based on feedback. Here is the feedback received:\nFeedback:\n\n\n\n---\n\n### Agents Description:\n{\'agents\': [{\'name\': \'Planner Agent\', \'role\': \'Planner Agent\', \'responsibilities\': [

## Step 6: Implementing the Agent

### Creating a Simple Agent Class

In this step, we define the `Agent` class, which is responsible for managing the state of the PM agent, interacting with the model, and processing the responses. This class encapsulates the core functionalities needed to execute tasks autonomously.

### Key Components:
- **Initialization (`__init__`)**: The constructor initializes the agent with its state, role, and model configuration. This setup is crucial for ensuring that the agent operates within the defined parameters and context.
  
- **Model Invocation (`invoke_model`)**: This method prepares the input payload, sends it to the model for processing, and handles the model's response. It's where the agent interacts with the LLM, using the system prompt and user prompt to generate meaningful outputs.

- **Task Execution (`execute_task`)**: This method allows the agent to execute a specific task based on a user request. It utilizes the system prompt tailored for the task and processes the response generated by the model.

The `Agent` class is fundamental in making our PM agent autonomous, enabling it to perform its duties without constant human oversight.

In [100]:
from langchain_core.messages.ai import AIMessage


class Agent:
    def __init__(self, state: AgentGraphState, role: str, model_config: dict):
        """
        Initialize the Agent with a state, role, and model configuration.
        """
        self.state = state
        self.role = role
        self.model_config = model_config

    def invoke_model(
        self,
        sys_prompt: str,
        user_prompt: str,
    ):
        """
        Prepare the payload, send the request to the model, and process the response.
        """
        # Prepare the payload
        payload = prepare_payload(user_prompt, sys_prompt, config=self.model_config)

        # Invoke the model and get the response
        response_json = request_model_generate_endpoint(
            payload, config=self.model_config
        )

        # Process the model's response
        response_content = process_model_response(response_json)

        # Return the processed response
        return response_content

    def execute_task(self, user_request: str, feedback: str = "") -> str:
        """
        Execute a simple task based on the user's request.
        """
        # Define a simple system prompt
        sys_prompt = write_pm_prompt(feedback_value=feedback)

        # Invoke the model with the user's request
        response = self.invoke_model(sys_prompt=sys_prompt, user_prompt=user_request)

        # Return the processed response
        return {"response": AIMessage(content=response)}

## Step 7: Validating Agent Output

### Ensuring Output Conforms to the Expected Structure

To maintain the reliability and consistency of the PM agent, it's crucial to validate the output generated by the agent. In this step, we define a schema that outlines the expected structure of the tasks created by the PM agent. This schema includes fields like `task_id`, `task_name`, `task_description`, `agent`, `status`, and others, ensuring that each task is well-defined and follows a standardized format.

### Validation Process:
- **Validation Function (`validate_agent_output`)**: This function checks the agent's output against the predefined schema. If the output conforms to the schema, it is marked as validated. Otherwise, it returns feedback indicating the issues, which can then be used to adjust the agent's behavior.

Validating the output ensures that the tasks generated by the PM agent are actionable and correctly formatted, which is essential for the smooth execution of the VM migration plan.

In [101]:
from jsonschema import validate, ValidationError

# Example schema for validating the agent's output
output_schema = {
    "type": "object",
    "properties": {
        "tasks": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "task_id": {
                        "type": "string",  # Unique identifier for the task.
                    },
                    "task_name": {
                        "type": "string",  # Short name of the task (e.g., "Validate VMware Access").
                    },
                    "task_description": {
                        "type": "string",  # Detailed description of the task to be executed.
                    },
                    "agent": {
                        "type": "string",
                        "enum": [
                            "architect",
                            "ocp_engineer",
                            "vsphere_engineer",
                            "networking",
                            "reviewer",
                            "cleanup",
                        ],  # Agent responsible for executing the task.
                    },
                    "status": {
                        "type": "string",
                        "enum": [
                            "pending",
                            "in_progress",
                            "completed",
                            "failed",
                        ],  # Current status of the task.
                    },
                    "dependencies": {
                        "type": "array",
                        "items": {
                            "type": "string",  # Task IDs that must be completed before this task can start.
                        },
                        "default": [],  # Default is an empty array if there are no dependencies.
                    },
                    "acceptance_criteria": {
                        "type": "string",  # Criteria that must be met to consider the task successfully completed.
                    },
                    "tool_to_use": {
                        "type": [
                            "string",
                            "null",
                        ],  # The tool the agent should use, or null if no tool is required.
                        "default": None,  # Default is None if no tool is specified.
                    },
                    "provided_inputs": {
                        "type": "object",  # Input data needed for this task.
                        "additionalProperties": {
                            "type": ["null", "string", "array"],
                            "items": {"type": "string"},
                        },
                        "default": {},  # Default is an empty object if no inputs are provided.
                    },
                },
                "required": [
                    "task_id",
                    "task_name",
                    "task_description",
                    "agent",
                    "status",
                    "acceptance_criteria",
                ],  # Dependencies, tool_to_use, and provided_inputs are optional.
            },
        }
    },
    "required": ["tasks"],
}

In [102]:
from langchain_core.messages import SystemMessage


def validate_agent_output(state: AgentGraphState, schema: dict = output_schema):
    """
    Validate the agent's output against the provided schema.
    """
    plan = state["response"]
    plan_content = plan[-1].content
    json_plan = json.loads(plan_content)
    try:
        validate(instance=json_plan, schema=schema)
        return {"validated": True, "feedback": SystemMessage(content="LGTM!")}
    except ValidationError as e:
        return {"validated": False, "feedback": SystemMessage(content=e.message)}

## Step 8: Workflow Execution Logic

### Determining Workflow Continuation

After validating the agent's output, we need to decide whether the workflow should continue or if it should conclude. This decision is based on the validation status of the agent's output.

### Key Function:
- **`should_continue`**: This function assesses whether the workflow should proceed based on whether the output was successfully validated. If validation is successful, the workflow ends; otherwise, it loops back to allow for adjustments and re-validation.

This logic ensures that only validated and correct outputs are used in the migration process, maintaining the integrity of the entire workflow.

In [103]:
from langgraph.graph import END

def should_continue(state):
    # Determine the next step based on the verification status

    if state["validated"]:
        return END
    else:
        return "pm"

## Step 9: Creating and Compiling the Workflow Graph

### Constructing the PM Agent's Workflow

In this step, we build the workflow graph that represents the PM agent's process. This graph outlines the sequence of operations, including task execution, validation, and decision-making.

### Key Components:
- **Node Definitions**: Each node in the graph represents a step in the workflow, such as invoking the PM agent or validating the output.
  
- **Edge Definitions**: Edges define the flow between nodes, determining how the agent progresses through the tasks and validation steps.

- **Workflow Compilation**: Once the graph is defined, it is compiled into a workflow that can be executed. This compiled workflow represents the full sequence of operations that the PM agent will follow to manage the VM migration.

By constructing and compiling this workflow graph, we ensure that the PM agent operates in a structured and efficient manner, handling tasks and making decisions in a logical sequence.

In [104]:
def pm_node_function(state: AgentGraphState):
    pm = Agent(
        state=state,
        role="PM",
        model_config=ollama_config,
    )

    if state["feedback"]:
        last_feedback = state["feedback"][-1].content
    else:
        last_feedback = None  # or some other default value

    return pm.execute_task(user_request=state["input"], feedback=last_feedback)

In [105]:
from langgraph.graph import StateGraph, START

def create_graph() -> StateGraph:
    """
    Create the state graph by defining nodes and edges.

    Returns:
    - StateGraph: The compiled state graph ready for execution.
    """
    graph = StateGraph(AgentGraphState)

    # Add nodes
    graph.add_node("pm", pm_node_function)
    graph.add_node("validate", lambda state: validate_agent_output(state=state))

    # Define the flow of the graph
    graph.add_edge(START, "pm")
    graph.add_edge("pm", "validate")
    graph.add_conditional_edges("validate", should_continue)
    # graph.add_edge("validate", END)

    return graph

### Initializing Sqlite Persistence for Graph State

In this cell, we define and use a method to initialize an `SqliteSaver` instance from the `langgraph.checkpoint.sqlite` module. The `SqliteSaver` class allows the graph state to be persisted in an SQLite database, which is more durable and suitable for applications requiring longer-term storage compared to an in-memory solution.

The `from_conn_stringx` method is defined as a class method that takes a connection string as input, creates a connection to the SQLite database using `sqlite3.connect`, and then returns an `SqliteSaver` instance using this connection. This method simplifies the creation of an `SqliteSaver` instance directly from a connection string.

This approach is particularly useful for ensuring that the state of the `StateGraph` is saved to a local or memory-based SQLite database, enabling the retention of context across multiple interactions in AI-driven applications.

In [106]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

def from_conn_stringx(cls, conn_string: str,) -> "SqliteSaver":
    return SqliteSaver(conn=sqlite3.connect(conn_string, check_same_thread=False))

SqliteSaver.from_conn_stringx=classmethod(from_conn_stringx)

memory = SqliteSaver.from_conn_stringx(":memory:")

## Step 10: Executing the Workflow

With the workflow graph compiled, the final step is to execute the workflow. This involves providing the agent with an input query, such as a request to generate a VM migration plan, and allowing the workflow to run through its defined sequence.

### Creating and Running the Workflow

In this step, we create and execute the PM agent's workflow to process a set of tasks.

- **Graph Creation**: 
  - We first create the workflow graph using `create_graph()` and compile it with a memory-based checkpoint.
  - The compiled workflow will manage the task execution, validation, and feedback handling.

- **Workflow Parameters**:
  - We define the number of iterations (`iterations = 10`), set verbose mode to `True`, and configure the thread ID.
  - A query containing three tasks (VM details retrieval, migration plan creation, and migration start) is provided as input.

- **Workflow Execution**:
  - The workflow is executed using `workflow.stream()`, and it processes each task sequentially.
  - Depending on the state of the workflow, feedback or task responses are printed to track progress.

This step runs the agent through the defined tasks and prints the state changes for each event in the workflow.

In [107]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory, interrupt_before=["validate"])
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
config = {"configurable": {"thread_id": "1"}}

Graph and workflow created.


### Running the Workflow: Initial Execution

- In the first step, we execute the workflow but stop **before validation**. This allows us to run the agent through the tasks up to the point where the output needs to be validated.
- The workflow processes the input tasks and prints the state changes, such as feedback or task responses from the PM agent.
- The execution stops before validation, allowing us to inspect the tasks generated so far.


In [108]:
query = "These are three tasks.  First task: The vSphere engineer will Retrieve the VM details of the vm called database. Don't forget the second Task: The OCP Engineer will Create a migration plan (vm_name: database) called 'database-plan'. Third task: The Ocp engineer will start the 'database-plan'. Just do this. Treat this as THREE differents tasks"
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
        if "validate" in event:
            event["validate"]["feedback"].pretty_print()
        elif "pm" in event:
            event["pm"]["response"].pretty_print()
        else:
            print("\nEvent:", event)
    else:
        print("\n")

================================== Ai Message ==================================

{
    "tasks": [
        {
            "task_id": "task_001",
            "task_name": "Retrieve VM Details (Database)",
            "task_description": "The vSphere engineer will retrieve the details of the virtual machine named 'database'.",
            "agent": "vsphere_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "VM details retrieved successfully.",
            "tool_to_use": "retrieve_vm_details_tool"
        },
        {
            "task_id": "task_002",
            "task_name": "Create Migration Plan (Database)",
            "task_description": "The OCP Engineer will create a migration plan (vm_name: database) called 'database-plan'.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": [
                "task_001"
            ],
            "acceptance_criteria": "Migration plan create

In [109]:
def custom_serializer(obj):
    if isinstance(obj, AIMessage):
        return {
            'content': obj.content,
            'id': obj.id,
            # Add other attributes as needed
        }
    elif isinstance(obj, SystemMessage):
        return {
            'content': obj.content,
            'id': obj.id,
            # Add other attributes as needed
        }
    # Add other custom class serializations here
    else:
        return str(obj)  # Fallback to string conversion if not recognized
    
state_snapshot = workflow.get_state(config)

print(json.dumps(state_snapshot.values, indent=4, default=custom_serializer))

print(state_snapshot.next)

{
    "input": "These are three tasks.  First task: The vSphere engineer will Retrieve the VM details of the vm called database. Don't forget the second Task: The OCP Engineer will Create a migration plan (vm_name: database) called 'database-plan'. Third task: The Ocp engineer will start the 'database-plan'. Just do this. Treat this as THREE differents tasks",
    "response": [
        {
            "content": "{\n    \"tasks\": [\n        {\n            \"task_id\": \"task_001\",\n            \"task_name\": \"Retrieve VM Details (Database)\",\n            \"task_description\": \"The vSphere engineer will retrieve the details of the virtual machine named 'database'.\",\n            \"agent\": \"vsphere_engineer\",\n            \"status\": \"pending\",\n            \"dependencies\": [],\n            \"acceptance_criteria\": \"VM details retrieved successfully.\",\n            \"tool_to_use\": \"retrieve_vm_details_tool\"\n        },\n        {\n            \"task_id\": \"task_002\",\n  

### Continuing the Workflow: Validation Phase

- After stopping before validation, we now run the workflow again to **continue the process and validate** the agent's output.
- During this phase, the workflow performs the validation of the task responses and prints feedback on whether the tasks meet the required criteria.
- This two-step execution allows us to break the workflow into distinct phases: task generation and output validation.

In [110]:
# Execute the workflow and print state changes
for event in workflow.stream(None, config):
    if verbose:
        if "validate" in event:
            event["validate"]["feedback"].pretty_print()
        elif "pm" in event:
            event["pm"]["response"].pretty_print()
        else:
            print("\nEvent:", event)
    else:
        print("\n")

================================ System Message ================================

LGTM!


In [111]:
state_snapshot = workflow.get_state(config)

print(json.dumps(state_snapshot.values, indent=4, default=custom_serializer))

{
    "input": "These are three tasks.  First task: The vSphere engineer will Retrieve the VM details of the vm called database. Don't forget the second Task: The OCP Engineer will Create a migration plan (vm_name: database) called 'database-plan'. Third task: The Ocp engineer will start the 'database-plan'. Just do this. Treat this as THREE differents tasks",
    "response": [
        {
            "content": "{\n    \"tasks\": [\n        {\n            \"task_id\": \"task_001\",\n            \"task_name\": \"Retrieve VM Details (Database)\",\n            \"task_description\": \"The vSphere engineer will retrieve the details of the virtual machine named 'database'.\",\n            \"agent\": \"vsphere_engineer\",\n            \"status\": \"pending\",\n            \"dependencies\": [],\n            \"acceptance_criteria\": \"VM details retrieved successfully.\",\n            \"tool_to_use\": \"retrieve_vm_details_tool\"\n        },\n        {\n            \"task_id\": \"task_002\",\n  

### Saving the Agent's Output to a File

In this step, we define a function, `save_json_to_file`, that allows us to save the generated JSON plan to a file. This function is important for persisting the agent's output for future reference or external use.

This step ensures that the final migration plan is saved in a persistent format for further analysis or use.

In [113]:
def save_json_to_file(json_plan, filename=""):
    try:
        # Save the content of json_plan to a file
        with open(filename, 'w') as file:
            json.dump(json_plan, file, indent=4)
        print(f"JSON saved successfully to {filename}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

if(state_snapshot.values["validated"]):
    print("The plan is validated!")
    json_plan = state_snapshot.values["response"][0].content
    content_dict = json.loads(json_plan)
    print(json.dumps(content_dict, indent=4))
    save_json_to_file(content_dict, "data/json_plan_output.json")
else:
    print("The plan is not validated!")

The plan is validated!
{
    "tasks": [
        {
            "task_id": "task_001",
            "task_name": "Retrieve VM Details (Database)",
            "task_description": "The vSphere engineer will retrieve the details of the virtual machine named 'database'.",
            "agent": "vsphere_engineer",
            "status": "pending",
            "dependencies": [],
            "acceptance_criteria": "VM details retrieved successfully.",
            "tool_to_use": "retrieve_vm_details_tool"
        },
        {
            "task_id": "task_002",
            "task_name": "Create Migration Plan (Database)",
            "task_description": "The OCP Engineer will create a migration plan (vm_name: database) called 'database-plan'.",
            "agent": "ocp_engineer",
            "status": "pending",
            "dependencies": [
                "task_001"
            ],
            "acceptance_criteria": "Migration plan created and validated.",
            "tool_to_use": "create_migra

## Conclusion

In this notebook, we've successfully implemented a PM agent capable of managing a VM migration plan autonomously. Through careful state management, effective use of system prompts, and a structured workflow graph, the agent can execute tasks, handle feedback, and adapt to changing requirements dynamically. This approach not only streamlines the migration process but also ensures that all tasks are executed efficiently and correctly.